In [1]:
import xarray as xr
import numpy as np
import functions as fun
import scipy.stats as st
import matplotlib.pyplot as plt

In [2]:
#mods = fun.models()
mods = ['bcc-csm1-1','CanESM2','CCSM4','CNRM-CM5','CSIRO-Mk3-6-0','GFDL-CM3','GISS-E2-R','HadGEM2-ES','inmcm4','IPSL-CM5A-LR','MIROC-ESM','MIROC5','MRI-CGCM3','NorESM1-M']

In [3]:
N = 500
scen = 'rcp85'

In [4]:
#Get 1x1 degree grid
with xr.open_dataset('../rawdata/rsl_85_2090.nc') as ds:
    lon = ds['x'].values
    lat = ds['y'].values
    mask = ds['slr_md'].values

#Get fingerprints
with xr.open_dataset(f'../data/fingerprints.nc') as ds:
    fp_a = ds['fp_a'].values
    fp_ea = ds['fp_ea'].values
    fp_wa = ds['fp_wa'].values
    fp_gr = ds['fp_gr'].values
    fp_gl = ds['fp_gl'].values

In [5]:
#Sterodynamic
gstdn = np.zeros(len(mods))
rstdn = np.zeros((len(mods),len(lat),len(lon)))

for m,model in enumerate(mods):
    #Get steric expansion    
    with xr.open_dataset(f'../data/steric/{model}_{scen}.nc') as ds:
        slr = ds['SLR'].values
        gstdn[m] = np.nanmean(slr[-20:])

    #Get zos
    with xr.open_dataset(f'../rawdata/zos/CorrectedZOS_EXP{scen}_{model}.nc') as ds:
        slr = ds['CorrectedZOS_reg'].values
        rstdn[m,:,:] = np.nanmean(slr[-20:,:,:],axis=0) + gstdn[m]
        rstdn[m,:,:][np.isnan(mask)] = np.nan

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: Mean of empty slice
  


In [6]:
#Glaciers
gglac = np.zeros((len(mods),N))
rglac = np.zeros((len(mods),len(lat),len(lon)))

for m,model in enumerate(mods):
    #Get global
    with xr.open_dataset(f'../data/glacier/{model}_{scen}.nc') as ds:
        slr = ds['SLR'].values
        gglac[m,:] = np.nanmean(slr[-20:])
    #Get regional
    for r in range(1,20):
        with xr.open_dataset(f'../data/glacier/{model}_{scen}_{r}.nc') as ds:
            slr = np.nanmean(ds['SLR'][-20:].values,axis=0)
            rglac[m,:,:] += slr*fp_gl[r-1,:,:]    

#Add process variance to global
std = np.nanstd(gglac)
gglac += np.random.normal(loc=0,scale=std,size=(len(mods),N))

In [7]:
#Greenland SMB
ggsmb = np.zeros((len(mods),N))
rgsmb = np.zeros((len(mods),N,len(lat),len(lon)))

for m,model in enumerate(mods):

    with xr.open_dataset(f'../rawdata/Gsmb_vars/{model}_{scen}.nc') as ds:
        TA = ds['TA'].values
        SF = ds['SF'].values
        years = ds['time'].values
    TA = TA[years<2101]
    SF = SF[years<2101]
    years = years[years<2101]            

    MW  = 84.2*TA + 2.4*TA**2. + 1.6*TA**3.
    SMB = SF-MW

    #Uncertainty following AR5
    F = np.exp(np.random.normal(loc=0.0, scale=0.4, size=N))
    E = np.random.uniform(1,1.15,size=N)
    for n in range(N):
        SLR = -np.cumsum(E[n]*F[n]*SMB)*2.8e-4 #in cm
        SLR = fun.detrend(SLR,np.array([]),years) #Reference to ref period
        ggsmb[m,n] = np.nanmean(SLR[-20:])
        rgsmb[m,n,:,:] = ggsmb[m,n]*fp_gr

In [8]:
#Antarctic SMB
gasmb = np.zeros((len(mods),N))
rasmb = np.zeros((len(mods),N,len(lat),len(lon)))

for m,model in enumerate(mods):

    with xr.open_dataset(f'../rawdata/Asmb_vars/{model}_{scen}.nc') as ds:
        TA = ds['TA'].values
        years = ds['time'].values

    TA = TA[years<2101]
    years = years[years<2101]
    #Uncertainty following AR5
    P = np.random.normal(loc=0.051, scale = 0.015,size=N)
    S = np.random.uniform(0,0.035,size=N)
    for n in range(N):
        SMB = 1923*P[n]*(1-S[n])*TA
        SMB[years<1995]=0
        SLR = -np.cumsum(SMB)*2.8e-4 #in cm
        gasmb[m,n] = np.nanmean(SLR[-20:])
        rasmb[m,n,:,:] = gasmb[m,n]*fp_a


In [9]:
#Antarctic dynamics
gadyn = np.zeros((len(mods),N))
radyn = np.zeros((len(mods),N,len(lat),len(lon)))

basin = ['eais','wedd','amun','ross','apen']
imods = ['AISM_VUB','BISI_LBL','CISM_NCA','FETI_VUB','GRIS_LSC','IMAU_VUB','ISSM_JPL','ISSM_UCI','MALI_LAN','PISM_AWI','PISM_DMI','PISM_PIK','PISM_VUW','PS3D_PSU','SICO_UHO','UA_UNN']

R = np.zeros((len(imods),len(basin),200))
for b,bb in enumerate(range(1,6)):
    for i,imod in enumerate(imods):
        with open(f'../rawdata/Larmip2019-master/RFunctions/RF_{imod}_BM08_R{bb}.dat') as f:
            R[i,b,:] = np.array([float(x) for x in f.readlines()])
            
for m,model in enumerate(mods):  

    with xr.open_dataset(f'../rawdata/Adyn_temp/{model}_{scen}.nc') as ds:
        TH = ds['TH'].values
        years = ds['time'].values

    #Uncertainty following Levermann et al 2020
    bms = np.random.uniform(7,16,N)
    ii  = np.random.randint(len(imods), size=N)

    for n in range(N):   
        for b,bb in enumerate(range(1,6)):
            SLR = np.zeros(len(years))
            for t,tt in enumerate(years):
                if t==0: continue
                SLR[t] = bms[n]*100*np.sum(TH[b,:t][::-1]*R[ii[n],b,:t])
            SLR = fun.detrend(SLR,np.array([]),years)
            gadyn[m,n] += np.nanmean(SLR[-20:])
            if b in [0,1,3]:
                radyn[m,n,:,:] += np.nanmean(SLR[-20:])*fp_ea
            else:
                radyn[m,n,:,:] += np.nanmean(SLR[-20:])*fp_wa

In [10]:
#Table 2 values
mstdn = np.zeros(len(mods))
mglac = np.zeros(len(mods))
mgsmb = np.zeros(len(mods))
masmb = np.zeros(len(mods))
madyn = np.zeros(len(mods))
for m,model in enumerate(mods):
    mstdn[m] = gstdn[m]
    mglac[m] = np.median(gglac[m,:])
    mgsmb[m] = np.median(ggsmb[m,:])
    masmb[m] = np.median(gasmb[m,:])
    madyn[m] = np.median(gadyn[m,:])

    print(f"{model:15} & {mstdn[m]:.0f} & {mglac[m]:.0f} & {mgsmb[m]:.0f} & {masmb[m]:.0f} & {madyn[m]:.0f} & {(mstdn[m]+mglac[m]+mgsmb[m]+masmb[m]+madyn[m]):.0f} \ ")
print(f"Multi-model mean & {np.mean(mstdn):.0f} & {np.mean(mglac):.0f} & {np.mean(mgsmb):.0f} & {np.mean(masmb):.0f} & {np.mean(madyn):.0f} & {np.mean(mstdn+mglac+mgsmb+masmb+madyn):.0f} \ ")
print(f'')
print(f"Variance (cm$^2$) & {np.var(mstdn):.0f} & {np.var(mglac):.0f} & {np.var(mgsmb):.0f} & {np.var(masmb):.0f} & {np.var(madyn):.0f} & {np.var(mstdn+mglac+mgsmb+masmb+madyn):.0f}")


bcc-csm1-1      & 24 & 20 & 3 & -4 & 3 & 46 \ 
CanESM2         & 29 & 23 & 9 & -6 & 9 & 64 \ 
CCSM4           & 28 & 20 & 1 & -5 & 13 & 56 \ 
CNRM-CM5        & 20 & 20 & 3 & -5 & 15 & 53 \ 
CSIRO-Mk3-6-0   & 28 & 15 & 5 & -5 & 5 & 48 \ 
GFDL-CM3        & 36 & 26 & 15 & -3 & 9 & 83 \ 
GISS-E2-R       & 28 & 15 & 4 & -3 & 9 & 52 \ 
HadGEM2-ES      & 26 & 25 & 8 & -6 & 10 & 64 \ 
inmcm4          & 23 & 10 & 1 & -4 & 11 & 42 \ 
IPSL-CM5A-LR    & 27 & 22 & 8 & -4 & 6 & 59 \ 
MIROC-ESM       & 34 & 23 & 13 & -5 & 5 & 69 \ 
MIROC5          & 28 & 22 & 8 & -3 & 10 & 65 \ 
MRI-CGCM3       & 21 & 16 & 3 & -4 & 7 & 43 \ 
NorESM1-M       & 28 & 20 & 4 & -3 & 10 & 59 \ 
Multi-model mean & 27 & 20 & 6 & -4 & 9 & 57 \ 

Variance (cm$^2$) & 17 & 18 & 17 & 1 & 10 & 120


In [11]:
#Save rsl ensemble
slr = np.nanmedian(np.repeat(rstdn[:,np.newaxis,:,:],N,axis=1)+np.repeat(rglac[:,np.newaxis,:,:],N,axis=1)+rgsmb+rasmb+radyn,axis=1)

slr2 = xr.DataArray(slr,dims=('model','lat','lon'),coords={'model':mods,'lat':lat,'lon':lon})
ds = xr.Dataset({'slr':slr2})
ds.to_netcdf(f'../data/ensemble_rsl_{scen}.nc')

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


In [12]:
#Save regional climate variance
vstdn = np.nanvar(rstdn,axis=0)
vglac = np.nanvar(rglac,axis=0)
vgsmb = np.nanvar(np.nanmedian(rgsmb,axis=1),axis=0)
vasmb = np.nanvar(np.nanmedian(rasmb,axis=1),axis=0)
vadyn = np.nanvar(np.nanmedian(radyn,axis=1),axis=0)
vslr = np.nanvar(rstdn+rglac+np.nanmedian(rgsmb,axis=1)+np.nanmedian(rasmb,axis=1)+np.nanmedian(radyn,axis=1),axis=0)
vres = vslr-vstdn-vglac-vgsmb-vasmb-vadyn

vstdn2 = xr.DataArray(vstdn,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
vglac2 = xr.DataArray(vglac,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
vgsmb2 = xr.DataArray(vgsmb,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
vasmb2 = xr.DataArray(vasmb,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
vadyn2 = xr.DataArray(vadyn,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
vres2 = xr.DataArray(vres,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
vslr2 = xr.DataArray(vslr,dims=('lat','lon'),coords={'lat':lat,'lon':lon})

ds = xr.Dataset({'slr':vslr2,'stdn':vstdn2,'glac':vglac2,'gsmb':vgsmb2,'asmb':vasmb2,'adyn':vadyn2,'res':vres2})
ds.to_netcdf(f'../data/climvar_{scen}.nc')

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Degrees of freedom <= 0 for slice.
  
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Degrees of freedom <= 0 for slice.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Degrees of freedom <= 0 for slice.
  after removing the cwd from sys.path.
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Degrees of freedom <= 0 for slice.
  """
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Degrees of freedom <= 0 for slice.
  
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Degrees of freedom <= 0 for slice.
  import sys


In [ ]:
#Save climate correlations

In [ ]:
#Save climate covariance

In [ ]:
#Save total correlations global

In [ ]:
#Save total correlations regional

In [ ]:
#Save total variance components